In [77]:
import numpy as np
import math
import tsplib95
import copy

In [78]:
problem = tsplib95.load('../data/bays29.tsp')

In [79]:
def nn_tsp(cities: list, weight: list) -> list:
    distance = 0
    i = 0
    start = list(cities.get_nodes())[0] - 1
    tour = [start]
    unvisited = list(cities.get_nodes())[0:28]
    weight_copy = copy.deepcopy(weight)
    
    while unvisited:
        masked_w = np.ma.masked_equal(weight_copy, 0, copy=True)
        
        C = np.min(masked_w[tour[i]])
        location = weight_copy[tour[i]].index(np.min(masked_w[tour[i]]))
        
        if (location in unvisited):
            weight_copy[location][tour[i]] = 0
            tour.append(location)
            distance += C
            unvisited.remove(location)
            i += 1
        else:
            weight_copy[tour[i]][location] = 0
    C = masked_w[tour[i]][0]
    distance += C
    tour.append(-1)

    return tour, distance

In [80]:
cities = problem
weight = cities.edge_weights

In [81]:
nn_tsp(cities, weight)

([0,
  27,
  5,
  11,
  8,
  4,
  25,
  28,
  2,
  1,
  20,
  19,
  9,
  3,
  14,
  17,
  13,
  21,
  16,
  10,
  18,
  15,
  26,
  23,
  7,
  22,
  24,
  6,
  12,
  -1],
 2258)

In [82]:
def cost_change(weight: list, n1: int, n2: int, n3: int, n4: int) -> int:
    return weight[n1][n3] + weight[n2][n4] - weight[n1][n2] - weight[n3][n4]

def two_opt(cities: list, weight: list, start: int) -> list:
    best = cities
    improved = True
    result = 0
    while improved:
        improved = False
        for i in range(1, len(cities) - 2):
            for j in range(i + 1, len(cities) - 2):
                if j - i == 1: continue
                if cost_change(weight, best[i-1], best[i], best[j-1], best[j]) < 0:
                    best[i:j] = best[j-1:i-1:-1]
                    improved = True
        cities = best
    
    for p in range(len(best) - 2):
        result += weight[best[p]][best[p+1]]
        
    last = best[p+1]
    result += weight[last][start]

    return best, result, start 

def nn_tsp_modified(cities: list, weight: list) -> list:
    distance = 0
    i = 0
    start = list(cities.get_nodes())[np.random.randint(0, 28)] - 1
    tour = [start]
    unvisited = list(cities.get_nodes())[0:28]
    weight_copy = copy.deepcopy(weight)

    if start != 0:
        unvisited.remove(start)
        unvisited.append(0)
    
    while unvisited:
        masked_w = np.ma.masked_equal(weight_copy, 0, copy=True)
        
        C = np.min(masked_w[tour[i]])
        location = weight_copy[tour[i]].index(np.min(masked_w[tour[i]]))
        
        if (location in unvisited):
            weight_copy[location][tour[i]] = 0
            tour.append(location)
            distance += C
            unvisited.remove(location)
            i += 1
        else:
            weight_copy[tour[i]][location] = 0
            
    C = masked_w[tour[i]][0]
    distance += C
    tour.append(-1)
    
    return two_opt(tour, weight, start)

In [83]:
nn_tsp_modified(cities, weight)

([13,
  17,
  14,
  3,
  9,
  19,
  1,
  2,
  28,
  25,
  4,
  8,
  11,
  5,
  27,
  0,
  20,
  12,
  23,
  7,
  26,
  22,
  15,
  18,
  24,
  6,
  10,
  21,
  16,
  -1],
 2082,
 13)